In [57]:
import pandas as pd
import nltk
from sklearn.metrics import classification_report
from tensorflow.python.keras.callbacks import EarlyStopping

nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras import backend as K

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ρωμανός\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
df = pd.read_csv(r'onion-or-not.csv',encoding='utf8')
pd.options.display.width = None
print(df.shape)
df.head(10)

(24000, 2)


,text,label
0,Entire Facebook Staff Laughs As Man Tightens P...,1
1,Muslim Woman Denied Soda Can for Fear She Coul...,0
2,Bold Move: Hulu Has Announced That They’re Gon...,1
3,Despondent Jeff Bezos Realizes He’ll Have To W...,1
4,"For men looking for great single women, online...",1
5,Kim Jong-Un Wonders If Nuclear Threats Distrac...,1
6,"Omaha dad finds pot brownies, eats 4 of them, ...",0
7,"Pokémon Go player stabbed, keeps playing",0
8,Job Placement Service Helps Students Who Fail ...,1
9,Idiot Zoo Animal With Zero Predators Still Pro...,1


1. Χωρίζουμε τους τίτλους σε λέξεις, δημιουργώντας ένα διάνυσμα λέξεων


In [59]:
df["tokens1"] = df["text"].apply(nltk.word_tokenize) #with punctuation
tokenizer = RegexpTokenizer(r'\w+')
df['tokens'] = df["text"].apply(tokenizer.tokenize) #without punctuation

df.head(10)

,text,label,tokens1,tokens
0,Entire Facebook Staff Laughs As Man Tightens P...,1,"[Entire, Facebook, Staff, Laughs, As, Man, Tig...","[Entire, Facebook, Staff, Laughs, As, Man, Tig..."
1,Muslim Woman Denied Soda Can for Fear She Coul...,0,"[Muslim, Woman, Denied, Soda, Can, for, Fear, ...","[Muslim, Woman, Denied, Soda, Can, for, Fear, ..."
2,Bold Move: Hulu Has Announced That They’re Gon...,1,"[Bold, Move, :, Hulu, Has, Announced, That, Th...","[Bold, Move, Hulu, Has, Announced, That, They,..."
3,Despondent Jeff Bezos Realizes He’ll Have To W...,1,"[Despondent, Jeff, Bezos, Realizes, He, ’, ll,...","[Despondent, Jeff, Bezos, Realizes, He, ll, Ha..."
4,"For men looking for great single women, online...",1,"[For, men, looking, for, great, single, women,...","[For, men, looking, for, great, single, women,..."
5,Kim Jong-Un Wonders If Nuclear Threats Distrac...,1,"[Kim, Jong-Un, Wonders, If, Nuclear, Threats, ...","[Kim, Jong, Un, Wonders, If, Nuclear, Threats,..."
6,"Omaha dad finds pot brownies, eats 4 of them, ...",0,"[Omaha, dad, finds, pot, brownies, ,, eats, 4,...","[Omaha, dad, finds, pot, brownies, eats, 4, of..."
7,"Pokémon Go player stabbed, keeps playing",0,"[Pokémon, Go, player, stabbed, ,, keeps, playing]","[Pokémon, Go, player, stabbed, keeps, playing]"
8,Job Placement Service Helps Students Who Fail ...,1,"[Job, Placement, Service, Helps, Students, Who...","[Job, Placement, Service, Helps, Students, Who..."
9,Idiot Zoo Animal With Zero Predators Still Pro...,1,"[Idiot, Zoo, Animal, With, Zero, Predators, St...","[Idiot, Zoo, Animal, With, Zero, Predators, St..."


2.	Από τις λέξεις αφαιρούμε τις καταλήξεις τους, κρατώντας μόνο το θέμα τους (stemming)


In [60]:
stemmer = PorterStemmer()
df['stemmed'] = df['tokens'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
df.head(10) # Print dataframe.

,text,label,tokens1,tokens,stemmed
0,Entire Facebook Staff Laughs As Man Tightens P...,1,"[Entire, Facebook, Staff, Laughs, As, Man, Tig...","[Entire, Facebook, Staff, Laughs, As, Man, Tig...","[entir, facebook, staff, laugh, As, man, tight..."
1,Muslim Woman Denied Soda Can for Fear She Coul...,0,"[Muslim, Woman, Denied, Soda, Can, for, Fear, ...","[Muslim, Woman, Denied, Soda, Can, for, Fear, ...","[muslim, woman, deni, soda, can, for, fear, sh..."
2,Bold Move: Hulu Has Announced That They’re Gon...,1,"[Bold, Move, :, Hulu, Has, Announced, That, Th...","[Bold, Move, Hulu, Has, Announced, That, They,...","[bold, move, hulu, ha, announc, that, they, re..."
3,Despondent Jeff Bezos Realizes He’ll Have To W...,1,"[Despondent, Jeff, Bezos, Realizes, He, ’, ll,...","[Despondent, Jeff, Bezos, Realizes, He, ll, Ha...","[despond, jeff, bezo, realiz, He, ll, have, To..."
4,"For men looking for great single women, online...",1,"[For, men, looking, for, great, single, women,...","[For, men, looking, for, great, single, women,...","[for, men, look, for, great, singl, women, onl..."
5,Kim Jong-Un Wonders If Nuclear Threats Distrac...,1,"[Kim, Jong-Un, Wonders, If, Nuclear, Threats, ...","[Kim, Jong, Un, Wonders, If, Nuclear, Threats,...","[kim, jong, Un, wonder, If, nuclear, threat, d..."
6,"Omaha dad finds pot brownies, eats 4 of them, ...",0,"[Omaha, dad, finds, pot, brownies, ,, eats, 4,...","[Omaha, dad, finds, pot, brownies, eats, 4, of...","[omaha, dad, find, pot, browni, eat, 4, of, th..."
7,"Pokémon Go player stabbed, keeps playing",0,"[Pokémon, Go, player, stabbed, ,, keeps, playing]","[Pokémon, Go, player, stabbed, keeps, playing]","[pokémon, Go, player, stab, keep, play]"
8,Job Placement Service Helps Students Who Fail ...,1,"[Job, Placement, Service, Helps, Students, Who...","[Job, Placement, Service, Helps, Students, Who...","[job, placement, servic, help, student, who, f..."
9,Idiot Zoo Animal With Zero Predators Still Pro...,1,"[Idiot, Zoo, Animal, With, Zero, Predators, St...","[Idiot, Zoo, Animal, With, Zero, Predators, St...","[idiot, zoo, anim, with, zero, predat, still, ..."


3. Αφαιρούμε από την συλλογή εκείνες τις λέξεις που είναι αρκετά κοινές και δεν προσφέρουν πληροφορία (stopwords removal)

In [61]:
df['without_stopwords'] = df['stemmed'].apply(lambda x: [item.lower() for item in x if item.lower() not in stop])
df.head(10)

,text,label,tokens1,tokens,stemmed,without_stopwords
0,Entire Facebook Staff Laughs As Man Tightens P...,1,"[Entire, Facebook, Staff, Laughs, As, Man, Tig...","[Entire, Facebook, Staff, Laughs, As, Man, Tig...","[entir, facebook, staff, laugh, As, man, tight...","[entir, facebook, staff, laugh, man, tighten, ..."
1,Muslim Woman Denied Soda Can for Fear She Coul...,0,"[Muslim, Woman, Denied, Soda, Can, for, Fear, ...","[Muslim, Woman, Denied, Soda, Can, for, Fear, ...","[muslim, woman, deni, soda, can, for, fear, sh...","[muslim, woman, deni, soda, fear, could, use, ..."
2,Bold Move: Hulu Has Announced That They’re Gon...,1,"[Bold, Move, :, Hulu, Has, Announced, That, Th...","[Bold, Move, Hulu, Has, Announced, That, They,...","[bold, move, hulu, ha, announc, that, they, re...","[bold, move, hulu, ha, announc, gonna, go, ahe..."
3,Despondent Jeff Bezos Realizes He’ll Have To W...,1,"[Despondent, Jeff, Bezos, Realizes, He, ’, ll,...","[Despondent, Jeff, Bezos, Realizes, He, ll, Ha...","[despond, jeff, bezo, realiz, He, ll, have, To...","[despond, jeff, bezo, realiz, work, 9, second,..."
4,"For men looking for great single women, online...",1,"[For, men, looking, for, great, single, women,...","[For, men, looking, for, great, single, women,...","[for, men, look, for, great, singl, women, onl...","[men, look, great, singl, women, onlin, date, ..."
5,Kim Jong-Un Wonders If Nuclear Threats Distrac...,1,"[Kim, Jong-Un, Wonders, If, Nuclear, Threats, ...","[Kim, Jong, Un, Wonders, If, Nuclear, Threats,...","[kim, jong, Un, wonder, If, nuclear, threat, d...","[kim, jong, un, wonder, nuclear, threat, distr..."
6,"Omaha dad finds pot brownies, eats 4 of them, ...",0,"[Omaha, dad, finds, pot, brownies, ,, eats, 4,...","[Omaha, dad, finds, pot, brownies, eats, 4, of...","[omaha, dad, find, pot, browni, eat, 4, of, th...","[omaha, dad, find, pot, browni, eat, 4, say, m..."
7,"Pokémon Go player stabbed, keeps playing",0,"[Pokémon, Go, player, stabbed, ,, keeps, playing]","[Pokémon, Go, player, stabbed, keeps, playing]","[pokémon, Go, player, stab, keep, play]","[pokémon, go, player, stab, keep, play]"
8,Job Placement Service Helps Students Who Fail ...,1,"[Job, Placement, Service, Helps, Students, Who...","[Job, Placement, Service, Helps, Students, Who...","[job, placement, servic, help, student, who, f...","[job, placement, servic, help, student, fail, ..."
9,Idiot Zoo Animal With Zero Predators Still Pro...,1,"[Idiot, Zoo, Animal, With, Zero, Predators, St...","[Idiot, Zoo, Animal, With, Zero, Predators, St...","[idiot, zoo, anim, with, zero, predat, still, ...","[idiot, zoo, anim, zero, predat, still, protec..."


4. αναθέτουμε ως βάρος την τιμή tf-idf

In [62]:
tf = TfidfVectorizer(lowercase='false', analyzer='word')
st = df['without_stopwords'].apply(lambda x: ' '.join(x))

re = tf.fit_transform(st)
response = tf.fit_transform(st).toarray()
print(re.shape)
print(re)
rows, cols = np.nonzero(response)

print(response[rows, cols])
print(response.shape)


(24000, 16521)
  (0, 13069)	0.30924352058001103
  (0, 11532)	0.4038006611301081
  (0, 14781)	0.5072544198071636
  (0, 9035)	0.1691140585248853
  (0, 8460)	0.3920273973386792
  (0, 13880)	0.35262233959490963
  (0, 5385)	0.2832299661866386
  (0, 5114)	0.3117044296347044
  (1, 15979)	0.400093851507405
  (1, 15567)	0.2659197854105327
  (1, 3628)	0.3025678760170136
  (1, 5504)	0.3818448434646714
  (1, 13610)	0.4565533569455909
  (1, 4200)	0.3601554234253247
  (1, 16216)	0.23435496864771768
  (1, 9853)	0.3725367438848746
  (2, 1159)	0.24411062532220687
  (2, 15940)	0.1991716368233393
  (2, 7368)	0.2726168304029761
  (2, 13344)	0.24251698633926413
  (2, 14015)	0.19753548579753888
  (2, 13273)	0.36706688407421084
  (2, 11982)	0.310222064605211
  (2, 843)	0.2691399593013618
  (2, 6346)	0.17897387591761013
  :	:
  (23996, 2819)	0.27446125275141814
  (23996, 5648)	0.2255999313831728
  (23996, 13942)	0.29017605329501134
  (23996, 4017)	0.20579527391561858
  (23996, 16220)	0.22033063330798958
  (23

In [63]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
X = response
Y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
print(X_train.shape)
model = Sequential()

model.add(Dense(16, activation='sigmoid', input_dim=16521))
model.add(Dense(16, activation='relu', input_dim=16521))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1_m,precision_m, recall_m])


(18000, 16521)


In [64]:
#Stop training when a monitored metric has stopped improving
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.1, verbose=1,mode='min')
history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=16,
                    callbacks=[early_stopping],
                    validation_data=(X_test,y_test),
                    verbose=1)

model.summary()
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
1125/1125 [==============================] - 11s 10ms/step - loss: 0.6468 - accuracy: 0.6119 - f1_m: 0.0374 - precision_m: 0.0410 - recall_m: 0.0605 - val_loss: 0.5898 - val_accuracy: 0.6470 - val_f1_m: 0.0622 - val_precision_m: 0.2107 - val_recall_m: 0.0376